In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft

# Downloading the phi3.5 and installing the finetuned phi3.5 
We first install the phi3.5 original model and then the bring in the adapter weights from the notebook where the model was originally finetuned. We then merge the adapter weights with the original model and obtained our model which can be used for the chatbot


In [12]:
base_model = "microsoft/Phi-3.5-mini-instruct"
#fine_tuned_model_1 = "/kaggle/input/phi4-finetuning-ecommerce-chatbot/finetuned_phi3_for_ecommerce_chatbot/"
fine_tuned_model = "/kaggle/input/finetuning-phi3-5-for-multiability-chatbot/Phi3.5-Combined-ChatBot/"
#fine_tuned_model_3 = "/kaggle/input/finetuning-phi3-5-telecom-chatbot/Phi3.5-Telecom-Chatbot/"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, LlamaTokenizer, Phi3ForCausalLM
from peft import PeftModel
import torch
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

#base_model_reload.resize_token_embeddings(128257)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [13]:
# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, fine_tuned_model)
model = model.merge_and_unload() #Merging the model with the adapter weights

In [25]:
messages = [
    {
        "role": "user",
        "content": "I was charged for 40 gb of data last month when I used only 30 hb. Please tell me how to resolve this?\n"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=250, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[0])

I was charged for 40 gb of data last month when I used only 30 hb. Please tell me how to resolve this?
 To address the issue of being charged for 40GB of data when you only used 30GB, please follow these steps:

1. Log in to your account on {{WEBSITE_URL}}.
2. Navigate to the {{DATA_SECTION}} section.
3. Select the {{CHECK_DATA_USAGE_OPTION}} option.
4. Review your usage details and locate the charges for the excess data.
5. Contact our customer support team through the {{SUPPORT_TEAM_CONTACT_METHOD}} to dispute the charges.
6. Provide any necessary information or documentation to support your claim.

Our team will review your request and take the necessary steps to resolve the issue. If you have any further questions, please feel free to reach out.


# Saving the model and pushing the model to hugging face

In [21]:
model_dir = "Phi-3.5-mini-instruct-Combined-Model"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir) #Saving the model to the kaggle output section 

('Phi-3.5-mini-instruct-Combined-Model/tokenizer_config.json',
 'Phi-3.5-mini-instruct-Combined-Model/special_tokens_map.json',
 'Phi-3.5-mini-instruct-Combined-Model/tokenizer.model',
 'Phi-3.5-mini-instruct-Combined-Model/added_tokens.json',
 'Phi-3.5-mini-instruct-Combined-Model/tokenizer.json')

In [ ]:
model.push_to_push(model_dir)    #Pushing the model to hugging face (please use hf_token before uploading the model)
tokenizer.push_to_hub(model_dir) #Pushing the tokenizer to hugging face (please use hf_token before uploading the model)